In [1]:
import numpy as np
import pandas as pd
import random
from random import randint

In [2]:
INIT_POPULATION = 200

In [3]:
df = pd.read_csv('16.txt', delimiter=' ')
parameters = df.columns.tolist()
df.columns = ['v', 'c']
df['w'] = df.index
data = df[['w','v','c']].values.tolist()
data = pd.DataFrame.from_records(data, columns=['w','v','c'])
W = int(parameters[0]) 
V = int(parameters[1]) 
genesCount = len(data) 

In [4]:
#создание начальной популяции
def createInitialPopulation():
    return [createIndividual() for _ in range(0, INIT_POPULATION)]

In [5]:
#создание особи
def createIndividual():
    individual = []
    for i in range(0, len(data)):
        individual.append(0)
    startIndex = randint(0, len(data) - 1)
    sumWeight = 0
    sumCapacity = 0
    for i in range(startIndex, len(data)):
        if (sumWeight + data.at[i,'w'] <= W) and (sumCapacity + data.at[i,'v'] <= V):
            individual[i] = 1
            sumWeight += data.at[i,'w']
            sumCapacity += data.at[i,'v']
    for i in range(0,startIndex):
        if (sumWeight + data.at[i,'w'] <= W) and (sumCapacity + data.at[i,'v'] <= V):
            individual[i] = 1
            sumWeight += data.at[i,'w']
            sumCapacity += data.at[i,'v']
    return individual

In [6]:
#стоимость особи
def individualC(individual):
    sumC = 0
    for i in range(0, genesCount):
        sumC += data.at[i,'c'] * individual[i]
    return sumC

In [7]:
#новая популяция
def newPopulation(population, childs):
    for individual in population:
        individual.append(individualC(individual) * 0.9)
    for child in childs:
        child.append(individualC(child))
    population.extend(childs)
    population = list(filter(lambda x: validateWandV(x), population))
    population.sort(key=lambda x: individualC(x), reverse=True)
    population = population[0:200]
    return population

In [8]:
#новый ребенок
def newChild(mom, dad):
    child = []
    mask = [randint(0, 1) for x in range(genesCount)]
    for i in range(0, len(mask)):
        if mask[i] == 1:
            child.append(mom[i])
        else:
            child.append(dad[i])
    return child

In [9]:
#скрещивание
def crossingover(population):
    childs = []
    while len(population) > 0:
        random.shuffle(population)
        mom = population.pop()
        if len(population) == 0:
            break
        dad = population.pop()
        child1 = newChild(mom, dad)
        child2 = newChild(mom, dad)
        if validateWandV(child1):
            childs.append(child1)
        if validateWandV(child2):
            childs.append(child2)
    return childs

In [10]:
#селекция
def selection(population):
    res_individuals = []
    individuals = [] 
    for i in range(len(population)):
        individuals.append(individualC(population[i]))
    
    max_fitness = max(individuals)
    min_fitness = min(individuals)

    for m in range(len(individuals)):
        if random.random() < individuals[m]/max_fitness:
            res_individuals.append(population[m])
    return res_individuals

In [11]:
#проверка на вес
def validateWandV(individual):
    sumW = 0
    sumV = 0
    for i in range (0, genesCount):
        sumW += data.at[i,'w'] * individual[i]
        sumV += data.at[i,'v'] * individual[i]
    if (sumW > W) or (sumV > V):
        return 0
    else: 
        return 1

In [12]:
#мутация
def mutation(individual):
    for i in range(0, len(individual)):
        if individual[i] == 0:
            individual[i] = 1
        else:
            individual[i] = 0
    return individual

In [13]:
#генетический алгоритм
def geneticAlgorithm():
    c = []
    Cpct = 0
    for i in range (0, len(data)):
        c.append(data.at[i,'c'])
    for i in range (0, len(c)):
        Cpct += c[i]
    minC = min(c)
    oldC = 0
    newC = Cpct

    MAX_ITERATION = 500
    population = createInitialPopulation()

    while MAX_ITERATION > 0:
        #Отбор
        population = selection(population)
        #Скрещивание
        childs = crossingover(population.copy())
        #Мутация
        rndIndividualInd = randint(0, len(population) - 1)
        population[rndIndividualInd] = mutation(population[rndIndividualInd])
        #Лучшая особь
        population = newPopulation(population, childs)
        bestIndividual = population[0]
        MAX_ITERATION -= 1
    return bestIndividual

In [14]:
bestIndividual = geneticAlgorithm()

sumW, sumV, sumC = 0, 0, 0
result = []
for i in range(genesCount):
    sumW += bestIndividual[i] * data.at[i,'w']
    sumV += bestIndividual[i] * data.at[i,'v']
    sumC += bestIndividual[i] * data.at[i,'c']
    if bestIndividual[i] == 1:
        result.append([data.at[i,'w'], data.at[i,'v'], data.at[i,'c']])
result_df = pd.DataFrame.from_records(result, columns=['w','v','c'])

In [15]:
print(result_df)
print("Вес:", sumW)
print("Объем:", round(sumV,1))
print("Ценность:", sumC)

         w    v      c
0   1160.0  0.8  372.0
1   1315.0  0.5  357.0
2    558.0  1.0  371.0
3   1139.0  1.0  288.0
4    151.0  0.4  286.0
5    774.0  0.6  171.0
6   1599.0  1.0  317.0
7    772.0  0.9  387.0
8    704.0  0.8  317.0
9   1372.0  0.8  239.0
10   721.0  1.0  231.0
11   976.0  0.6  300.0
12   378.0  1.0  217.0
13   823.0  0.9  367.0
14   491.0  0.6  232.0
Вес: 12933.0
Объем: 11.9
Ценность: 4452.0


In [16]:
file=open('4.2res.txt','w', encoding='utf-8')
file.write(result_df.to_string())
file.write("\nВес: " + str(sumW))
file.write("\nОбъем: " + str(round(sumV,1)))
file.write("\nЦенность: " + str(sumC))
file.close()